In [2]:
import pandas as pd 
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import tabulate

2023-01-16 22:44:41.579332: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-16 22:44:42.122539: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 22:44:42.122588: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-16 22:44:43.893886: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-1

In [131]:
#importing the data, list of movie corresponding to the order of rating in y_target
df_movie_features_ML = pd.read_pickle("ml-latest-small/df_movie_features_ML.pkl")
df_movie_mapping = df_movie_features_ML[["title", "rated_movies_avg"]]
df_movie_mapping.reset_index(drop=True, inplace=True)
#features and target data
movie_features = np.load("ml-latest-small/movie_features_ML.npy", allow_pickle=True)
user_features = np.load("ml-latest-small/user_features_ML.npy", allow_pickle=True)
y_target = np.load("ml-latest-small/y_target_ML.npy", allow_pickle=True)

,title,rated_movies_avg
0,Toy Story (1995),3.920930
1,Grumpier Old Men (1995),3.259615
2,Heat (1995),3.946079
3,Seven (a.k.a. Se7en) (1995),3.975369
4,"Usual Suspects, The (1995)",4.237745
...,...,...
57886,Split (2017),3.333333
57887,John Wick: Chapter Two (2017),4.142857
57888,Get Out (2017),3.633333
57889,Logan (2017),4.280000


In [112]:
# scale training data
movie_features_unscaled = movie_features
user_features_unscaled = user_features
y_train_unscaled    = y_target

scalerMovie = StandardScaler()
scalerMovie.fit(movie_features)
movie_features_norm = scalerMovie.transform(movie_features)

scalerUser = StandardScaler()
scalerUser.fit(user_features)
user_features_norm = scalerUser.transform(user_features)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_target.reshape(-1, 1))
y_target_norm = scalerTarget.transform(y_target.reshape(-1, 1))
#ynorm_test = scalerTarget.transform(y_test.reshape(-1, 1))

In [113]:
movie_features_norm.shape, user_features_norm.shape, y_target_norm.shape

((57891, 21), (57891, 21), (57891, 1))

In [114]:
#spiliting the data for training and testing

movie_train, movie_test = train_test_split(movie_features_norm, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_features_norm, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test = train_test_split(y_target_norm, train_size=0.80, shuffle=True, random_state=1)

In [115]:
# constructing the model
num_user_features = user_features_norm.shape[1] 
num_item_features = movie_features_norm.shape[1]
num_outputs = 64
tf.random.set_seed(1)
user_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
  
    ### END CODE HERE ###  
])

item_NN = tf.keras.models.Sequential([
    ### START CODE HERE ###     
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs)
  
  
    ### END CODE HERE ###  
])

# create the user input and point to the base network
input_user = tf.keras.layers.Input(shape=(num_user_features))
vu = user_NN(input_user)
vu = tf.linalg.l2_normalize(vu, axis=1) ### this is a function to normalize output vectors produced by NNs. This is done in order to ensure that the dot product between
### the user and item is a measure of the similarity between them.  

# create the item input and point to the base network
input_item = tf.keras.layers.Input(shape=(num_item_features))
vm = item_NN(input_item)
vm = tf.linalg.l2_normalize(vm, axis=1)

# compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

model.summary()

Model: "model_10"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 21)]         0           []                               
                                                                                                  
 input_22 (InputLayer)          [(None, 21)]         0           []                               
                                                                                                  
 sequential_20 (Sequential)     (None, 64)           46784       ['input_21[0][0]']               
                                                                                                  
 sequential_21 (Sequential)     (None, 64)           46784       ['input_22[0][0]']               
                                                                                           

In [116]:
###Compiling the model

tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.02)
model.compile(optimizer=opt,
              loss=cost_fn)

In [117]:
#fiting the model

tf.random.set_seed(1)
model.fit([user_train, movie_train], y_train, epochs=30)

Epoch 1/30
1448/1448 [==============================] - 7s 3ms/step - loss: 0.1692
Epoch 2/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1623
Epoch 3/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1609
Epoch 4/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1595
Epoch 5/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1576
Epoch 6/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1567
Epoch 7/30
1448/1448 [==============================] - 5s 4ms/step - loss: 0.1555
Epoch 8/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1543
Epoch 9/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1526
Epoch 10/30
1448/1448 [==============================] - 5s 3ms/step - loss: 0.1514
Epoch 11/30
1448/1448 [==============================] - 5s 4ms/step - loss: 0.1501
Epoch 12/30
1448/1448 [==============================] - 6s 4ms/step - loss: 0.1487
E

In [118]:
model.evaluate([user_test, movie_test], y_test)

362/362 [==============================] - 1s 2ms/step - loss: 0.1473


0.14725254476070404

In [119]:
#evaluating with root mean sqaure error
pred = model.predict([user_test, movie_test])

from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(y_test, pred))
print(f'Root mean sqaure error is {rmse}')

362/362 [==============================] - 1s 1ms/step
Root mean sqaure error is 0.3837350156233624


In [9]:
def mean_from_ratings(lst):
    tmp = 0
    counter = 0
    for value in lst:
        if value > 0:
            tmp += value
            counter +=1
    return  tmp/counter
    

###Creating a new profile.

In [138]:
Adventure = 4.0
Animation = 1.0
Children = 1.0
Comedy = 4.0
Fantasy = 4.0
Romance = 1.0
Drama = 1.0
Action = 5.0
Crime = 5.0
Thriller = 5.0
Horror = 4.0
Mystery = 5.0
SciFi = 5.0
War = 1.0
Musical = 1.0
Documentary = 1.0
IMAX_1 = 0.0
Western = 1.0
FilmNoir = 0.0
no_genres_listed = 0.0
lst_genres = [Adventure,
                    Animation,
                    Children,
                    Comedy,
                    Fantasy,
                    Romance,
                    Drama,
                    Action,
                    Crime,
                    Thriller,
                    Horror,
                    Mystery,
                    SciFi,
                    War,
                    Musical,
                    Documentary,
                    IMAX_1,
                    Western,
                    FilmNoir,
                    no_genres_listed]
                    
user_avg = mean_from_ratings(lst_genres)
#user_count = df_user_features["user_count"].mean()

user_vec = np.array([[Adventure,
                    Animation,
                    Children,
                    Comedy,
                    Fantasy,
                    Romance,
                    Drama,
                    Action,
                    Crime,
                    Thriller,
                    Horror,
                    Mystery,
                    SciFi,
                    War,
                    Musical,
                    Documentary,
                    IMAX_1,
                    Western,
                    FilmNoir,
                    no_genres_listed,
                    user_avg,]])

In [139]:
user_vec

array([[4.        , 1.        , 1.        , 4.        , 4.        ,
        1.        , 1.        , 5.        , 5.        , 5.        ,
        4.        , 5.        , 5.        , 1.        , 1.        ,
        1.        , 0.        , 1.        , 0.        , 0.        ,
        2.88235294]])

In [140]:
# generate and replicate the user vector to match the number movies in the data set.


item_vecs = movie_features
user_vecs = np.tile(user_vec, (len(item_vecs), 1))

# scale our user and item vectors
suser_vecs = scalerUser.transform(user_vecs)
sitem_vecs = scalerMovie.transform(item_vecs)

# make a prediction
y_p = model.predict([suser_vecs, sitem_vecs])

# unscale y prediction 
y_pu = scalerTarget.inverse_transform(y_p)

1810/1810 [==============================] - 3s 2ms/step


In [141]:
#matching the resilts to titles

df_y_pu = pd.DataFrame(y_pu)
df_new_user_ratings= pd.merge(df_y_pu, df_movie_mapping, left_index=True, right_index=True)
df_new_user_ratings_sorted = df_new_user_ratings.sort_values(0, ascending =False)
df_new_user_ratings_sorted.drop_duplicates("title").head(30)#.to_csv("recomdation_05")


,0,title,rated_movies_avg
30187,3.973523,"Dark Knight Rises, The (2012)",3.993421
8942,3.911807,Harry Potter and the Half-Blood Prince (2009),3.887931
16126,3.906764,"Dark Knight, The (2008)",4.238255
50560,3.873777,Hot Fuzz (2007),4.000000
48929,3.852846,The Lego Movie (2014),3.870968
46935,3.851189,Enchanted (2007),3.676471
5124,3.841525,Annihilation (2018),3.833333
45057,3.828609,Captain America: The First Avenger (2011),3.546875
43479,3.828609,X-Men: First Class (2011),3.790698
29048,3.820051,V for Vendetta (2006),3.885000
